This notebook performs the optimization task to find the location where a new grocery store should be placed, to maximize access

How this works (implementation):

1. Use the helper_population_allocation.py to allocate a population count to each residential building (not ready yet, some placeholder function in there)
2. Use the helper_distance_calculation.py to calculate existing access and distance between a residential and commercial building
3. Once 1 and 2 are done, all parameters are ready. 

4. Then this notebook does some pre optimization setup and then runs an optimization model in Gurobi. It's pretty anti climatic. 

Next steps:

- Update the population allocation helper file
- Figure out the correct ordering of arrays. It's not accurate currently 


In [81]:
# Import libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
import gurobipy as gp
from gurobipy import GRB

# Helper modules
import helper_population_allocation as pa
import helper_distance_calculation as dc

# Avoid printing set copy warnings
import warnings
warnings.filterwarnings("ignore")



PRE-OPTIMIZATION SETUP

In [82]:
%%time

# Get the main buildings dataset 
buildings_df = gpd.read_file('../processed_data/relevant_buildings.shp')

# Create ID variable
buildings_df.reset_index(drop=True, inplace=True)
buildings_df['building_id'] = buildings_df.index + 1
buildings_df['building_id'] = buildings_df.apply(lambda row: str(row['building_id']) + '-' + str(row['CLASS']) , axis=1)

# buildings_df = buildings_df.sample(n=2000, random_state=1)  # Remove later




Wall time: 8.03 s


In [83]:
# Create arrays to track ordering (residential)
res_buildings = buildings_df[buildings_df['class_reco'].str.contains('Residential')]
res_buildings = res_buildings.sort_values('building_id')
res_buildings = dc.get_geocoordinate(res_buildings, 'geometry')

res_buildings_array = np.array(res_buildings['building_id'])
res_buildings_coordinates_array = np.array(res_buildings['coordinates'])

In [84]:
# Create arrays to track ordering (Commercial)
comm_buildings = buildings_df[buildings_df['class_reco'].str.contains('commercial')]
comm_buildings = comm_buildings.sort_values('building_id')
comm_buildings = dc.get_geocoordinate(comm_buildings, 'geometry')

comm_buildings_array = np.array(comm_buildings['building_id'])
comm_buildings_coordinates_array = np.array(comm_buildings['coordinates'])


In [85]:
# Create arrays to track ordering (grocery stores)
grocery_stores = buildings_df[buildings_df['class_reco'].str.contains('Grocery')]
grocery_stores = grocery_stores.sort_values('building_id')
grocery_stores = dc.get_geocoordinate(grocery_stores, 'geometry')

grocery_stores_array = np.array(grocery_stores['building_id'])
grocery_stores_coordinates_array = np.array(grocery_stores['coordinates'])


In [86]:
%%time
# DONT RUN THIS AGAIN

# Create parameter matrices (Res comm access matrix - Bij)
# [i,j] value indicates whether residential building i is within access distance of commercial building j
# res_comm_distance_matrix, res_comm_access_matrix = dc.calculate_access(res_buildings_coordinates_array, comm_buildings_coordinates_array)

# # Save file
# np.save('res_comm_access_matrix', res_comm_access_matrix)
# np.save('res_comm_distance_matrix', res_comm_distance_matrix)


Wall time: 0 ns


In [87]:
# Load the files and use it 
res_comm_access_matrix = np.load('res_comm_access_matrix.npy')
res_comm_distance_matrix = np.load('res_comm_distance_matrix.npy')

In [88]:
%%time

# Create parameter matrices (Res groc access array - Aj)
# ith value indicates whether the ith residential building has existing access
res_groc_distance_matrix, res_groc_access_matrix = dc.calculate_access(res_buildings_coordinates_array, grocery_stores_coordinates_array)
res_access_array = np.amax(res_groc_access_matrix, 1)


Wall time: 55.1 s


In [89]:
%%time

# Create parameter matrices (Res Population - Pj)
# ith value indicates the population in the ith column
res_population = pa.get_population(geopandas_dataframe=res_buildings) 
res_population_array = np.array(res_population['population'])
res_population_array

Wall time: 1.95 s


array([1, 1, 1, ..., 1, 2, 2], dtype=int64)

OPTIMIZATION

Steps:
i = set of all commercial buildings
j = set of all residential buildings

Decision variable:
- Ci = 1 if the new grocery store is put in commercial building i, 0 otherwise

Parameters:
- Aj = 1 if residential building j already has access to a food store (within 1 mile)
- Bij = 1 if commercial building i is within 1 mile of residential building j
- Pj = Population at building j

Objective function:

Max $$ \sum_i \sum_j (1-A_j)*P_j*B_{ij}*C_i$$

Constraint (only 1 grocery store location being allocated):
$$ \sum_i C_i = 1$$





In [90]:
# Gurobi takes forever to optimize since it relies on for loops. So we are going to do the optimization manually. 

###########################
# STEP 1: Take the res_comm_access_matrix, remove those rows (each row represents a residential building) which have existing access
###########################
existing_access_indices = res_access_array.nonzero()[0] # These are indices of residential buildings that currently have access
res_comm_access_matrix_subset = np.delete(res_comm_access_matrix, existing_access_indices, axis=0 )

###########################
# STEP 2: Do the same thing for res_population_array so that the ordering matches
###########################
res_population_array_sub = np.delete(res_population_array, existing_access_indices, axis=0)

###########################
# STEP 3: Do a matrix multiplication between res_population_array_sub and res_comm_access_matrix_sub
###########################

# How this works:

# 1. Reshape res_population_array_sub to be (1 * 2780) 2D array
# 2. res_comm_access_matrix_sub is (2780 * 6895)
# 3. When you do matrix multiplication of 1 and 2, you get a (1*6895) array
# 4. Each element of this array would represent the sum of the population at each residential building multiplied by whether that residential building and that particular commercial building
# are within access region. So for example, first element of this result would be P0 * whether res building 0 and comm building 0 are within access + P1 * whether res building 1 and comm building 0 are within access and so on
# So each element of the result represents the total new population that would gain access if a commercial building is put at that index

res_population_array_sub = np.reshape(res_population_array_sub, (-1, len(res_population_array_sub)))
new_access_array = np.matmul(res_population_array_sub, res_comm_access_matrix_subset)

###########################
# STEP 4: Print results
###########################
chosen_comm_index = np.argmax(new_access_array)
chosen_comm_building = comm_buildings_array[chosen_comm_index]
new_access_created = np.max(new_access_array)

print(f"the new store should be put at commercial building {chosen_comm_building}")
print(f"putting the store here would give access to {new_access_created} new people")


the new store should be put at commercial building 2816-R
putting the store here would give access to 2723 new people


In [134]:
# Now write the optimization that can put n stores in a greedy way

# Process - do the exact same thing as above, but in a loop. Keep updating the access array as you place each commercial building

def place_n_stores(num_stores):

    store_indices = []
    store_ids = []
    marginal_access_gain = []

    res_access_array_copy = res_access_array.copy()
    for n in range(num_stores):

        if np.sum(res_access_array_copy) == len(res_buildings_array): # This means every building now has access
            pass
        else:
            # STEP 1
            existing_access_indices = res_access_array_copy.nonzero()[0] # These are indices of residential buildings that currently have access
            res_comm_access_matrix_subset = np.delete(res_comm_access_matrix, existing_access_indices, axis=0 )

            # STEP 2
            res_population_array_sub = np.delete(res_population_array, existing_access_indices, axis=0)

            # STEP 3
            res_population_array_sub = np.reshape(res_population_array_sub, (-1, len(res_population_array_sub)))
            new_access_array = np.matmul(res_population_array_sub, res_comm_access_matrix_subset)

            chosen_comm_index = np.argmax(new_access_array)
            chosen_comm_building = comm_buildings_array[chosen_comm_index]
            new_access_created = np.max(new_access_array)

            # STEP 4: Update results and arrays
            
            # Which residential buildings does this new store give access to
            new_buildings_with_access = res_comm_access_matrix[:,chosen_comm_index].nonzero()[0]    # These are the indices in the res_access_array that need to be replaced (these buildings now have access)

            # Update the access values of these buildings in the access array
            replace_vals = list(np.ones(new_buildings_with_access.shape)) # These are the values which with certain elements of the access array will be replaced with

            # Perform replace
            res_access_array_copy[new_buildings_with_access] = replace_vals

            # STEP 5: Store results
            store_indices.append(chosen_comm_index)
            store_ids.append(chosen_comm_building)
            marginal_access_gain.append(new_access_created)


    return store_indices, store_ids, marginal_access_gain





In [135]:
store_indices, store_ids, marginal_access_gain = place_n_stores(5)

print(f"Num additional stores needed to fill complete access = {len(store_indices)}")

results_df = pd.DataFrame(list(zip(store_ids, marginal_access_gain)),
                                columns = ['building_id', 'marginal_access_gain'])
                                
results_df


Num additional stores needed to fill complete access = 2


,building_id,marginal_access_gain
0,2816-R,2723
1,116276-C,880


In [69]:
# # GUROBI SUCKS, DON'T RUN

# ########################################
# # SET UP MODEL
# ########################################

# m = gp.Model("food_access")

# num_commercial_buildings = len(comm_buildings_array)
# num_residential_buildings = len(res_buildings_array)


# ########################################
# # ASSIGN DECISION VARIABLES
# ########################################

# c_i = m.addVars(range(num_commercial_buildings), vtype=GRB.BINARY)

# print('decision vars added')

# #######################################
# # OBJECTIVE FUNCTION
# ########################################

# m.setObjective(sum(((1-res_access_array[j]) 
#                         * res_population_array[j] 
#                         * res_comm_access_matrix.T[i,j]
#                         * c_i[i] 
#                     for j in range(num_residential_buildings) 
#                 for i in range(num_commercial_buildings))))
# m.modelSense = GRB.MAXIMIZE

# print('objective function set')

# ########################################
# # CONSTRAINTS
# ########################################

# m.addConstr(sum(c_i[i] for i in range(num_commercial_buildings)) ==  1) 

# print('constraints added')
